In [1]:
#init
import sys
import os
from dotenv import load_dotenv
import requests
import json

load_dotenv()

sys.path.append(os.getenv('FUNCTION_PATH'))
from crawl_func import get_page, get_page_selenium
from database_func import store_provider_collection, store_review_collection

gartner_overview_review_url = ["https://www.gartner.com/reviews/api2-proxy/reviews/market/vendor/filter?vendorSeoName=amazon-web-services&marketSeoName=ml-infrastructure-as-a-service&productSeoName=amazon-ec2-p3-instances&startIndex=1&endIndex=30&filters=%7B%22products%22%3A%5B%5D%2C%22reviewRating%22%3A%5B%5D%2C%22partnerReview%22%3A%5B%5D%2C%22companySize%22%3A%5B%5D%2C%22industry%22%3A%5B%5D%2C%22deploymentRegion%22%3A%5B%5D%2C%22function%22%3A%5B%5D%2C%22tags%22%3A%5B%5D%7D&sort=-helpfulness", 
                               "https://www.gartner.com/reviews/api2-proxy/reviews/market/vendor/filter?vendorSeoName=amazon-web-services&marketSeoName=cloud-database-management-systems&productSeoName=amazon-rds&startIndex=1&endIndex=30&filters=%7B%22products%22%3A%5B%5D%2C%22reviewRating%22%3A%5B%5D%2C%22partnerReview%22%3A%5B%5D%2C%22companySize%22%3A%5B%5D%2C%22industry%22%3A%5B%5D%2C%22deploymentRegion%22%3A%5B%5D%2C%22function%22%3A%5B%5D%2C%22tags%22%3A%5B%5D%7D&sort=-helpfulness",
                               "https://www.gartner.com/reviews/api2-proxy/reviews/market/vendor/filter?vendorSeoName=amazon-web-services&marketSeoName=application-platforms-reviews&productSeoName=aws-lambda&startIndex=1&endIndex=30&filters=%7B%22products%22%3A%5B%5D%2C%22reviewRating%22%3A%5B%5D%2C%22partnerReview%22%3A%5B%5D%2C%22companySize%22%3A%5B%5D%2C%22industry%22%3A%5B%5D%2C%22deploymentRegion%22%3A%5B%5D%2C%22function%22%3A%5B%5D%2C%22tags%22%3A%5B%5D%7D&sort=-helpfulness", 
                               "https://www.gartner.com/reviews/api2-proxy/reviews/market/vendor/filter?vendorSeoName=amazon-web-services&marketSeoName=container-management&productSeoName=amazon-elastic-kubernetes-service-amazon-eks&startIndex=1&endIndex=15&filters=%7B%22products%22%3A%5B%5D%2C%22reviewRating%22%3A%5B%5D%2C%22partnerReview%22%3A%5B%5D%2C%22companySize%22%3A%5B%5D%2C%22industry%22%3A%5B%5D%2C%22deploymentRegion%22%3A%5B%5D%2C%22function%22%3A%5B%5D%2C%22tags%22%3A%5B%5D%7D&sort=-helpfulness", 
                               "https://www.gartner.com/reviews/api2-proxy/reviews/market/vendor/filter?vendorSeoName=google&marketSeoName=content-collaboration-tools&productSeoName=google-drive&startIndex=1&endIndex=30&filters=%7B%22products%22%3A%5B%5D%2C%22reviewRating%22%3A%5B%5D%2C%22partnerReview%22%3A%5B%5D%2C%22companySize%22%3A%5B%5D%2C%22industry%22%3A%5B%5D%2C%22deploymentRegion%22%3A%5B%5D%2C%22function%22%3A%5B%5D%2C%22tags%22%3A%5B%5D%7D&sort=-helpfulness", 
                               "https://www.gartner.com/reviews/api2-proxy/reviews/market/vendor/filter?vendorSeoName=google&marketSeoName=strategic-cloud-platform-services&productSeoName=google-cloud-platform&startIndex=1&endIndex=15&filters=%7B%22products%22%3A%5B%5D%2C%22reviewRating%22%3A%5B%5D%2C%22partnerReview%22%3A%5B%5D%2C%22companySize%22%3A%5B%5D%2C%22industry%22%3A%5B%5D%2C%22deploymentRegion%22%3A%5B%5D%2C%22function%22%3A%5B%5D%2C%22tags%22%3A%5B%5D%7D&sort=-helpfulness", 
                               "https://www.gartner.com/reviews/api2-proxy/reviews/market/vendor/filter?vendorSeoName=google&marketSeoName=multiexperience-development-platforms&productSeoName=firebase&startIndex=1&endIndex=15&filters=%7B%22products%22%3A%5B%5D%2C%22reviewRating%22%3A%5B%5D%2C%22partnerReview%22%3A%5B%5D%2C%22companySize%22%3A%5B%5D%2C%22industry%22%3A%5B%5D%2C%22deploymentRegion%22%3A%5B%5D%2C%22function%22%3A%5B%5D%2C%22tags%22%3A%5B%5D%7D&sort=-helpfulness", 
                               "https://www.gartner.com/reviews/api2-proxy/reviews/market/vendor/filter?vendorSeoName=google&marketSeoName=container-management&productSeoName=google-kubernetes-engine-gke&startIndex=1&endIndex=15&filters=%7B%22products%22%3A%5B%5D%2C%22reviewRating%22%3A%5B%5D%2C%22partnerReview%22%3A%5B%5D%2C%22companySize%22%3A%5B%5D%2C%22industry%22%3A%5B%5D%2C%22deploymentRegion%22%3A%5B%5D%2C%22function%22%3A%5B%5D%2C%22tags%22%3A%5B%5D%7D&sort=-helpfulness", 
                               "https://www.gartner.com/reviews/api2-proxy/reviews/market/vendor/filter?vendorSeoName=microsoft&marketSeoName=container-management&productSeoName=azure-kubernetes-service-aks&startIndex=1&endIndex=15&filters=%7B%22products%22%3A%5B%5D%2C%22reviewRating%22%3A%5B%5D%2C%22partnerReview%22%3A%5B%5D%2C%22companySize%22%3A%5B%5D%2C%22industry%22%3A%5B%5D%2C%22deploymentRegion%22%3A%5B%5D%2C%22function%22%3A%5B%5D%2C%22tags%22%3A%5B%5D%7D&sort=-helpfulness", 
                               "https://www.gartner.com/reviews/api2-proxy/reviews/market/vendor/filter?vendorSeoName=alibaba-cloud&marketSeoName=container-management&productSeoName=alibaba-cloud-container-service-for-kubernetes-ack&startIndex=1&endIndex=15&filters=%7B%22products%22%3A%5B%5D%2C%22reviewRating%22%3A%5B%5D%2C%22partnerReview%22%3A%5B%5D%2C%22companySize%22%3A%5B%5D%2C%22industry%22%3A%5B%5D%2C%22deploymentRegion%22%3A%5B%5D%2C%22function%22%3A%5B%5D%2C%22tags%22%3A%5B%5D%7D&sort=-helpfulness", 
                                ]

gartner_detail_review_url = ["https://www.gartner.com/reviews/market/ml-infrastructure-as-a-service/vendor/amazon-web-services/product/amazon-ec2-p3-instances/review/view/", 
                              "https://www.gartner.com/reviews/market/cloud-database-management-systems/vendor/amazon-web-services/product/amazon-rds/review/view/", 
                              "https://www.gartner.com/reviews/market/application-platforms-reviews/vendor/amazon-web-services/product/aws-lambda/review/view/", 
                              "https://www.gartner.com/reviews/market/container-management/vendor/amazon-web-services/product/amazon-elastic-kubernetes-service-amazon-eks/review/view/", 
                              "https://www.gartner.com/reviews/market/content-collaboration-tools/vendor/google/product/google-drive/review/view/", 
                              "https://www.gartner.com/reviews/market/strategic-cloud-platform-services/vendor/google/product/google-cloud-platform/review/view/", 
                              "https://www.gartner.com/reviews/market/multiexperience-development-platforms/vendor/google/product/firebase/review/view/", 
                              "https://www.gartner.com/reviews/market/container-management/vendor/google/product/google-kubernetes-engine-gke/review/view/", 
                              "https://www.gartner.com/reviews/market/container-management/vendor/microsoft/product/azure-kubernetes-service-aks/review/view/", 
                              "https://www.gartner.com/reviews/market/strategic-cloud-platform-services/vendor/alibaba-cloud/product/alibaba-cloud/review/view/"]

getapp_review_url = ["https://www.getapp.com/it-management-software/a/ec2/reviews/", 
                     "https://www.getapp.com/it-management-software/a/amazon-rds/reviews/", 
                     "https://www.getapp.com/it-management-software/a/aws-cloudtrail/reviews/",
                     "https://www.getapp.com/collaboration-software/a/google-drive/reviews/", 
                     "https://www.getapp.com/it-management-software/a/google-cloud-platform/reviews/", 
                     "https://www.getapp.com/it-management-software/a/alibaba-cloud/reviews/", 
                     "https://www.getapp.com/it-management-software/a/azure/reviews/"]

In [7]:
#define function to get review from getapp, gartner
def count_star_get_app(soup):
    empty_star = soup.find_all("i", class_="Stars_fadedStar__asCL5")
    star = soup.find_all("i", class_="Stars_activeStar__xGADt")
    
    if(len(empty_star) + len(star) == 5):
        return len(star)
    else:
        return len(star) + 0.5

def get_provider(soup):
    if("Amazon" in soup or "AWS" in soup):
        return "aws"
    elif("Google" in soup):
        return "google"
    elif("Microsoft" in soup or "Azure" in soup):
        return "microsoft"
    elif("Alibaba" in soup):
        return "alibaba"
    else:
        return "other"
    
def get_product(soup):
    if("CloudTrail" in soup):
        return ["CloudTrail", "Cloud Platform", "Security"]
    elif("Kubernetes" in soup):
        return ["Kubernetes Engine", "Cloud Platform", "Container Management"]
    elif("EC2" in soup):
        return ["EC2", "Cloud Platform", "Machine Learning"]
    elif("RDS" in soup):
        return ["RDS", "Cloud Platform", "Database"]
    elif("Drive" in soup):
        return ["Drive", "Cloud Platform"]
    elif("GCP" in soup):
        return ["GCP", "Cloud Platform"]
    elif("Azure" in soup):
        return ["Azure", "Cloud Platform", "Container Management"]
    elif("Cloud" in soup or "Microsoft Azure" in soup):
        return "Cloud Platform"
    elif("DB" in soup or "Database" in soup):
        return ["Database", "Cloud Platform"]
    else:
        return "Other"

In [ ]:
#Get review from GetApp
for url in getapp_review_url:
    soup = get_page_selenium(url)
    
    rating_criteria = soup.find_all("li", class_="RatingCriteriaItem CriteriaItem_root__w97kp RatingCriteria_item__7V2yC")
    
    overall_rating = soup.find("p", class_="Typography Typography_root__FajAY Typography_display-sm__hPPKe RatingDisplay_number__akybV").text
    value_for_money = rating_criteria[0].find("div", class_="Grid Stars Stars_ratingStarsContainer__z_69j CriteriaItem_ratingAlignment__KpAt5")
    ease_of_use = rating_criteria[1].find("div", class_="Grid Stars Stars_ratingStarsContainer__z_69j CriteriaItem_ratingAlignment__KpAt5")
    features = rating_criteria[2].find("div", class_="Grid Stars Stars_ratingStarsContainer__z_69j CriteriaItem_ratingAlignment__KpAt5")
    customer_support = rating_criteria[3].find("div", class_="Grid Stars Stars_ratingStarsContainer__z_69j CriteriaItem_ratingAlignment__KpAt5")
    likelihood_to_recommend = soup.find("span", class_="RatingCriteria_likelihood__Zw7_0").text

    provider_container = soup.find("p", class_="Typography Typography_root__FajAY Typography_heading-sm__0XjfO Link Link_root__4foJ9 ListingInfo_listingName__sroeE ListingInfo_listingNameNoLink__qlm90").text
    provider = get_provider(provider_container)
    product = get_product(provider_container)

    review = [{
        "cloud_provider": provider,
        "product": product,
        "overall_rating": overall_rating,
        "likelihood_to_recommend": likelihood_to_recommend,
        "country": None,
        "region": None,
        "review_url": url,
        "criteria_ratings": {
            "value_for_money": count_star_get_app(value_for_money),
            "ease_of_use": count_star_get_app(ease_of_use),
            "features": count_star_get_app(features),
            "customer_support": count_star_get_app(customer_support)
        }
        
    }]
    store_review_collection(review)


In [ ]:
#Get review from Gartner
for overview_url, detailed_url in zip(gartner_overview_review_url, gartner_detail_review_url):
    soup = get_page_selenium(overview_url)
    page_content = soup.find("div", hidden=True).text
    json_data = json.loads(page_content)
    reviews = json_data["userReviews"]
    for review in reviews:
        url = detailed_url + str(review["reviewId"])
        review_soup_page = get_page_selenium(url).find("div", id="review")
        # Extract the overall rating
        overall_rating = review_soup_page.find("div", class_="category review-rating").find("div", class_="review-pdf-star-data").text.strip()

        # Extract the criteria ratings
        criteria_rating_key = review_soup_page.find_all("div", class_="col-xs-6 col-sm-6 col-md-7 sect-name")
        criteria_rating_value = review_soup_page.find_all("div", class_="section-rating-pdf-data")

        # Extract the country
        country_elem = review_soup_page.find("ul", {"data-key": "deployment-country-multi"})
        country = country_elem.text.strip() if country_elem else None

        # Extract the region
        region_elem = review_soup_page.find("ul", {"data-key": "deployment-region"})
        region = region_elem.text.strip() if region_elem else None

        # Extract product and provider
        product_container = review_soup_page.find("span", {"itemprop": "itemReviewed"}).text.strip()
        provider = get_provider(product_container)
        product = get_product(product_container)

        # Create a dictionary for the criteria ratings
        criteria_ratings = {}
        for key, value in zip(criteria_rating_key, criteria_rating_value):
            criteria_ratings[key.text.strip()] = value.text.strip()

        # Build the review dictionary
        review = [{
            "cloud_provider": provider,
            "product": product,
            "overall_rating": overall_rating,
            "likelihood_to_recommend": None,
            "country": country,
            "region": region,
            "review_url": url,
            "criteria_ratings": criteria_ratings
        }]
        # print(review)
        store_review_collection(review)

In [ ]:

# "Evaluation & Contracting"
#     "Ability to understand your organization's needs"
#     "Timely and complete response to product questions"
#     "Pricing and contract flexibility (pricing and terms)"

# "Integration & Deployment"
#     "Ease of integration using standard APIs and tools"
#     "Availability of quality 3rd-party resources (integrators, service providers, etc.)"
#     "Quality and availability of end-user training"
#     "Ease of deployment"

# "service support"
#     "Timeliness of vendor's response"
#     "Quality of technical support"
#     "Quality of peer user community"
# for review in reviews:
#         url = aws_ec2_gartner_review_url + str(review["reviewId"])
#         review_soup_page = get_page_selenium(url).find("div", id="review")
#         # Extract the overall rating
#         overall_rating = review_soup_page.find("div", class_="category review-rating").find("div", class_="review-pdf-star-data").text.strip()

#         # Extract the criteria ratings
#         criteria_rating_key = review_soup_page.find_all("div", class_="col-xs-6 col-sm-6 col-md-7 sect-name")
#         criteria_rating_value = review_soup_page.find_all("div", class_="section-rating-pdf-data")

#         # Extract the country
#         country_elem = review_soup_page.find("ul", {"data-key": "deployment-country-multi"})
#         country = country_elem.text.strip() if country_elem else None

#         # Extract the region
#         region_elem = review_soup_page.find("ul", {"data-key": "deployment-region"})
#         region = region_elem.text.strip() if region_elem else None

#         # Extract product and provider
#         product_container = review_soup_page.find("span", {"itemprop": "itemReviewed"}).text.strip()
#         provider = get_provider(product_container)
#         product = get_product(product_container)

#         # Create a dictionary for the criteria ratings
#         criteria_ratings = {}
#         for key, value in zip(criteria_rating_key, criteria_rating_value):
#             criteria_ratings[key.text.strip()] = value.text.strip()

#         # Build the review dictionary
#         review = [{
#             "cloud_provider": provider,
#             "product": product,
#             "overall_rating": overall_rating,
#             "likelihood_to_recommend": None,
#             "country": country,
#             "region": region,
#             "review_url": url,
#             "criteria_ratings": criteria_ratings
#         }]
#         # print(review)
#         store_review_collection(review)